We can create a store of all the data so that we can quickly access it. This took about half an hour to run on my machine.

In [1]:
import pandas as pd

In [ ]:
store = pd.HDFStore('store.h5')
months={4:'April', 5:'May', 6:'June', 7:'July',8:'August', 9:'September'}

for m in months.keys():
    for y in range(2001, 2016):
        path = '../../data/CHARLOTTE/BigBOX/'
        def dateparse(Y, m, d, H, M):
            d = pd.datetime(int(Y), int(m), int(d), int(H), int(M))
            return d

        df = pd.read_csv(path+'Charlotte_box_radar_{yyyy}_{mm:02d}.csv'.format(yyyy=y, mm=m),
                         header=None, sep = ',', na_values = '-99',
                         parse_dates={'date_time': [0,1,2,3,4]},
                         date_parser=dateparse, index_col=[0])
        df.columns = range(0,19600)
        store['CHARLOTTE_{yyyy}_{mm:02d}'.format(yyyy=y, mm=m)] = df
store.close()

We can also store computed values that we feel we will be accessing a lot. This one takes a while too. 

In [2]:
from radar import *

In [ ]:
months={4:'April', 5:'May', 6:'June', 7:'July',8:'August', 9:'September'}
monthly_mean = {}

for m in months.keys():
    to_stack = []
    for y in range(2001, 2016):
        c = Radar('Charlotte', t=pd.datetime(y, m, 4), how='hdf5')
        to_stack.append(np.nanmean(c.box, axis=0)*4)
    monthly_mean.update({m: np.nanmean(np.stack(to_stack), axis=0)})

In [ ]:
computed = pd.HDFStore('computed.h5')
monthly_mean_df = pd.DataFrame([v.flatten() for v in monthly_mean.values()], index=monthly_mean.keys())
computed['monthly_mean_df'] = monthly_mean_df
computed.close()

Once we have the store of all the data it is pretty quick to make this other store of the big events. Now we will be able to load really quickly.

In [3]:
TOP50 = pd.read_csv('../../Paper/TOP50_Events/TOP50_events_Charlotte.csv', parse_dates=[0,1])

In [ ]:
for n in range(0,50):
    top = pd.HDFStore('TOP50.h5')
    store = pd.HDFStore('store.h5')
    y = TOP50.t_begin[n].year
    m = TOP50.t_begin[n].month
    d = TOP50.t_begin[n].day
    city='charlotte'
    fname = '{c}_{yyyy}_{mm:02d}'.format(c=city.upper(), yyyy=y, mm=m)
    df = store[fname]
    top['storm_{yyyy}_{mm:02d}_{dd:02d}'.format(yyyy=y, mm=m, dd=d)] = df[TOP50.t_begin[n]:TOP50.t_end[n]]
    top.close()
    store.close()

In [ ]:
for n in range(0,50):
    top=pd.HDFStore('TOP50.h5')
    y = TOP50.t_begin[n].year
    m = TOP50.t_begin[n].month
    d = TOP50.t_begin[n].day
    df = top['storm_{yyyy}_{mm:02d}_{dd:02d}'.format(yyyy=y, mm=m, dd=d)]
    top.close()
    
    
    computed = pd.HDFStore('computed.h5')
    computed = storm_meandf.mean(axis=1)
    monthly_mean_df = pd.DataFrame([v.flatten() for v in monthly_mean.values()], index=monthly_mean.keys())
    computed['monthly_mean_df'] = monthly_mean_df
    computed.close()